In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

# import os
# os.environ['ARRAY_MODULE'] = 'numpy'
from gigablochs import bloch, rf
from gigablochs import xp, asnumpy
from gigablochs import animation

In [ ]:
duration = 0.08 # seconds
dt = 0.00001 # seconds

flip_angle = 180 # degrees
rf_duration = 0.011 # seconds
rf_bandwidth = 1600 # Hz
rf_stretch = 2.75
rf_amplitude = 3e-5 # T

off_resonance = 1600 # Hz
spectrum_lines = 600
B1_inhomogeneity = np.linspace(0.01, 1, 100) # fraction of B1

T1 = 1.5 # seconds
T2 = 0.1 # seconds

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse_am, rf_pulse_fm = rf.adiabatic_pulse(flip_angle, rf_duration,
                                              rf_bandwidth, rf_stretch, dt, amplitude=rf_amplitude)
rf_am = rf.extend(rf_pulse_am, duration, dt)
rf_fm = rf.extend(rf_pulse_fm, duration, dt)

dfz = np.linspace(0, off_resonance, spectrum_lines)
B = bloch.construct_B_field(rf_am, rf_fm=rf_fm, off_resonance=dfz, B1_sensitivity=B1_inhomogeneity)

In [ ]:
B.size * 4 / 1e9 # GB

In [ ]:
B.shape

In [ ]:
type(B)

In [ ]:
k = rf.adiabaticity(rf_pulse_am, rf_pulse_fm, dt)

In [ ]:
k.min()

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='Adiabatic RF Pulse')

plot.line(rf_time * 1e3, rf_pulse_am * 1e6, line_width=2, legend_label='AM: sech')
plot.yaxis.axis_label = 'RF Amplitude (µT)'
plot.y_range = bkm.Range1d(start=0, end=rf_amplitude * 1.1e6)
plot.extra_y_ranges['freq'] = bkm.Range1d(start=-rf_bandwidth * 0.6, end=rf_bandwidth * 0.6)
naxis = bkm.LinearAxis(y_range_name='freq', axis_label='Frequency (Hz)')
plot.add_layout(naxis, 'right')
plot.line(rf_time * 1e3, rf_pulse_fm, line_width=2, y_range_name='freq', color='green', legend_label='FM: tanh')

plot.extra_y_ranges['k'] = bkm.Range1d(start=0, end=2)
kaxis = bkm.LinearAxis(y_range_name='k', axis_label='log(adiabaticity)')
plot.add_layout(kaxis, 'right')
plot.line(rf_time * 1e3, np.log10(k), line_width=2, y_range_name='k', color='pink', legend_label='log(k)')

plot.xaxis.axis_label = 'Time (ms)'
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(time, asnumpy(B[:, 0, -1, 0] * 1e6), line_width=2, legend_label='Bx', alpha=0.5)
plot.line(time, asnumpy(B[:, 0, -1, 1] * 1e6), line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(time, asnumpy(B[:, 0, -1, 2] * 1e6), line_width=2, legend_label='Bz', color='green')
plot.line(time, np.linalg.norm(asnumpy(B[:, 0, -1] * 1e6), axis=-1), line_width=2, legend_label='|B|', color='purple')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B (µT)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
mags = bloch.sim(B, T1, T2, duration, dt)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(time, asnumpy(mags[:, 0, -1, 0]), line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, asnumpy(mags[:, 0, -1, 1]), line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, asnumpy(mags[:, 0, -1, 2]), line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[1], end // 10):
    alpha = 1 - offres / end
    plot.line(time, asnumpy(mags[:, offres, -1, 2]), line_width=2, legend_label=f'{dfz[offres]:g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (a.u.)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
plot = bkp.figure(width=800, height=400, title=f'Flipped Magnetization Spectrum for {rf_bandwidth} Hz bandwidth')
plot.line(asnumpy(dfz), asnumpy(mags.min(axis=0)[:, -1, -1]), line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
bkp.show(plot)

In [ ]:
plot = bkp.figure(width=800, height=400, title=f'Flipped Magnetization Spectrum for {rf_bandwidth} Hz bandwidth')
plot.line(asnumpy(B1_inhomogeneity), asnumpy(mags.min(axis=0)[0, :, -1]), line_width=2)
plot.xaxis.axis_label = 'B1 Inhomogeneity'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.y_range = bkm.DataRange1d(start=-1, end=1)
bkp.show(plot)

In [ ]:
freq = asnumpy(dfz)
b1 = asnumpy(B1_inhomogeneity)
minmag = asnumpy(mags.min(axis=0)[..., -1].T)

title = 'Inverted Magnetization Spectrum'
plot = bkp.figure(width=1000, height=500, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=-1, high=1)
image = plot.image([minmag], y=[b1.min()], x=[freq.min()],
                   dh=[b1.max() - b1.min()],
                   dw=[freq.max() - freq.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'B1 Inhomogeneity'
plot.x_range = bkm.DataRange1d(start=freq.min(), end=freq.max())
plot.y_range = bkm.DataRange1d(start=b1.min(), end=b1.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = 'Min Magnetization (ref M0)'
plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
mags.shape

In [ ]:
N = np.nonzero(time == 0.012)[0][0]

In [ ]:
B.shape

In [ ]:
Beff = asnumpy(B[:N, 0, -1])
magnets = asnumpy(mags[:N, 0, -1])
ts = time[:N]

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(ts, asnumpy(Beff[..., 0]), line_width=2, legend_label='Bx', alpha=0.5)
plot.line(ts, asnumpy(Beff[..., 1]), line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(ts, asnumpy(Beff[..., 2]), line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B (µT)'
plot.x_range = bkm.DataRange1d(start=0, end=ts[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
np.arctan2(Beff[0, 0], Beff[0, -1]) * 180 / np.pi # initial angle degrees

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(ts, asnumpy(magnets[..., 0]), line_width=2, legend_label='Mx', alpha=0.5)
plot.line(ts, asnumpy(magnets[..., 1]), line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(ts, asnumpy(magnets[..., 2]), line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=ts[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
ts.shape

## Animation

In [ ]:
def rescale(arr, ogtime, newtime):
    # return sig.resample(arr, newtime.size, axis=0, window='hann')
    # return sig.resample_poly(arr, 1, ogtime.size // newtime.size, axis=0, padtype='maximum')
    return arr[::ogtime.size // newtime.size]

In [ ]:
anim_duration = 15 # s
sample_rate = 16 # Hz

anim_time = np.arange(0, anim_duration, 1/sample_rate)
anim_m = rescale(asnumpy(magnets), ts, anim_time)
anim_b = rescale(asnumpy(Beff), ts, anim_time)

ts.size / anim_time.size

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Beff')
plot.line(anim_time, anim_b[..., 0], line_width=2, legend_label='Bx', alpha=0.5)
plot.line(anim_time, anim_b[..., 1], line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(anim_time, anim_b[..., 2], line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B (µT)'
plot.x_range = bkm.DataRange1d(start=0, end=anim_time[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(anim_time, anim_m[..., 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(anim_time, anim_m[..., 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(anim_time, anim_m[..., 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=anim_time[-1])
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
speed = animation.speed(ts, anim_time)
speed

In [ ]:
animation.bloch_sphere(anim_m, anim_b, np.gradient(anim_time), speed,
                       quality='high_quality')

In [ ]:
# np.savez_compressed('adiabatic_mags.npz', mags=anim_m, beff=anim_b, time=anim_time, sample_rate=sample_rate, speed=speed)